<a href="https://colab.research.google.com/github/Harshitha-Nuthikadi/movie-recommender-system/blob/main/movireRS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
movies=pd.read_csv('tmdb_5000_movies.csv')
credits=pd.read_csv('tmdb_5000_credits.csv',on_bad_lines='skip',engine='python')

FileNotFoundError: [Errno 2] No such file or directory: 'tmdb_5000_movies.csv'

In [ ]:
movies.head(1)

In [ ]:
credits.head(1)

In [ ]:
movies = movies.merge(credits,on='title')
movies.head(1)

In [ ]:
# genres
# id
# keywords
# title
# overview
# cast

# crew
movies=movies[['movie_id','title','overview','genres','keywords','cast','crew']]
movies.head()

In [ ]:
movies.isnull().sum()

In [ ]:
movies.dropna(inplace=True)
movies.duplicated().sum()

In [ ]:

import ast
def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L


In [ ]:
# movies.dropna(inplace=True)
movies['genres'] = movies['genres'].apply(convert)
movies['keywords']=movies['keywords'].apply(convert)
movies.head()


In [ ]:

def convert3(text):
    L = []
    counter = 0
    for i in ast.literal_eval(text):
        if counter != 3:
            L.append(i['name'])
            counter+=1
        else:
            break
    return L

In [ ]:

movies['cast'] = movies['cast'].apply(convert)
movies['cast'] = movies['cast'].apply(lambda x:x[0:3])

movies.head()

In [ ]:
def fetch_director(text):
    L = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L

In [ ]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [ ]:
movies.head(1)

In [ ]:
movies['overview'][0]

In [ ]:
# converting this in to list so that we can cancatinate
movies['overview']=movies['overview'].apply(lambda x:x.split())
movies.head(2)

In [ ]:
# transformation ->remove spaces
movies['genres']=movies['genres'].apply(lambda x:[i.replace(' ','') for i in x])
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(' ','') for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(' ','') for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(' ','') for i in x])
movies.head(2)


In [ ]:
movies['tags']=movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']

In [ ]:
movies.head()

In [ ]:
new_df=movies[['movie_id','title','tags']]
new_df.head()

In [ ]:
new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))


In [ ]:
new_df.head(2)

In [ ]:
new_df['tags'][0]

In [ ]:
new_df['tags']=new_df['tags'].apply(lambda x:x.lower())
new_df.head()

In [ ]:
new_df['tags'][0]

In [ ]:
import sklearn

from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words='english')


In [ ]:
vectors=cv.fit_transform(new_df['tags']).toarray()
vectors

In [ ]:
import nltk


In [ ]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()


In [ ]:
def stem(text):
  y=[]
  for i in text.split():
    y.append(ps.stem(i))
  return " ".join(y)

In [ ]:
ps.stem('loving')

In [ ]:
new_df['tags'][0]

In [ ]:
stem('in the 22nd century, a paraplegic marine is dispatched to the moon pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. action adventure fantasy sciencefiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d samworthington zoesaldana sigourneyweaver jamescameron')

In [ ]:
new_df['tags']=new_df['tags'].apply(stem)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
similarity=cosine_similarity(vectors)


In [ ]:

def recommend(movie):
    index = new_df[new_df['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(new_df.iloc[i[0]].title)


In [ ]:
new_df[new_df['title']=='Avatar'].index[0]

In [ ]:
recommend('Batman Begins')

In [ ]:
import pickle


In [ ]:
pickle.dump(new_df,open('movies.pkl','wb'))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pickle.dump(new_df.to_dict(),open('movie_dict.pkl','wb'))

In [ ]:
pickle.dump(similarity,open('similarity.pkl','wb'))

In [ ]:
import pickle
movies_dict = pickle.load(open('movie_dict.pkl', 'rb'))
import pandas as pd
movies = pd.DataFrame(movies_dict)
print(movies.head())
